In [59]:
import pickle as pkl
import torch
from collections import Counter

In [2]:
file_train = '../../dataset/dataset_graph_full_150step_larger_train.pkl'
with open(file_train, 'rb') as f:
    cont = pkl.load(f)

In [3]:
print("Training episodes: {}".format(len(cont)))

Training episodes: 6738


In [4]:
names_preds = '../../results_inference/detfull_encoder_task_graph/pred_dict.pkl'
with open(names_preds, 'rb') as f:
    name_preds = pkl.load(f)
    

In [58]:
def get_goal(file_name):
    new_file_name = file_name.replace('.pik', '_reduced.pik')

    with open(new_file_name, 'rb') as f:
        resp = pkl.load(f)
    if not resp['valid']:
        return None
    gt_task = resp['gt_task_graph']
    init_task = resp['task_graph_time'][0]
    diff_task = gt_task - init_task
    task_str = ''
    if 'table' in resp['task_name']:
        
        for i in range(136):
            goal_str = '{}.{}'.format(name_preds[i][0], name_preds[i][1])
            if diff_task[i] > 0:
                task_str += '{}: {}\n'.format(goal_str, diff_task[i])
    return (task_str, new_file_name, 'table' in resp['task_name'], resp['task_name'])

In [62]:
file_train = '../../dataset/dataset_graph_full_150step_larger_train.pkl'
with open(file_train, 'rb') as f:
    cont = pkl.load(f)
task_names = []
file_names = list(cont.keys())
for filename in file_names:
    resp = get_goal(filename)
    if resp is not None:
        task_names.append(resp[-1])

In [63]:
Counter(task_names)

Counter({'put_fridge': 1534,
         'watch_tv': 1426,
         'setup_table': 1584,
         'prepare_food': 1200,
         'put_dishwasher': 989})

In [6]:
filename = '/data/vision/torralba/frames/data_acquisition/SyntheticStories/online_wah/agent_preferences/dataloader/../dataset_episodes_noscratch/data_structured/train_env_task_set_150_full_task.all/1_full_opencost0_closecostFalse_walkcost0.05_forgetrate0/logs_episode.2251_iter.0.pik'
with open(filename, 'rb') as f:
    cont = pkl.load(f)
    
resp = get_goal(filename)
if resp is not None and resp[2]:
    print(resp[1])
    print(resp[0])

/data/vision/torralba/frames/data_acquisition/SyntheticStories/online_wah/agent_preferences/dataloader/../dataset_episodes_noscratch/data_structured/train_env_task_set_150_full_task.all/1_full_opencost0_closecostFalse_walkcost0.05_forgetrate0/logs_episode.2251_iter.0_reduced.pik
cutleryfork.kitchentable: 3
plate.kitchentable: 2
waterglass.kitchentable: 3



In [8]:
import sys
sys.path.append('../../utils')
import utils_rl_agent

Using backend: pytorch


In [16]:
it = 0
graph_helper = utils_rl_agent.GraphHelper(max_num_objects=100, 
                                          toy_dataset=False)
graph_info, _ = graph_helper.build_graph_for_task(
                    cont['graph'][it], 
                    character_id=1, include_edges=True, 
                    obs_ids=cont['obs'][it], relative_coords=False,
                    unique_from=True)


task_graph = graph_helper.build_task_graph(graph_info)

it = 101

graph_info, _ = graph_helper.build_graph_for_task(
                    cont['graph'][it], 
                    character_id=1, include_edges=True, 
                    obs_ids=cont['obs'][it], relative_coords=False,
                    unique_from=True)

task_graph_end = graph_helper.build_task_graph(graph_info)
                


In [51]:
a1 = list(np.maximum(np.zeros(task_graph.shape), task_graph_end - task_graph).astype(np.int32))
ab = list(np.maximum(np.zeros(task_graph.shape), task_graph_end).astype(np.int32))
a2 = list(task_graph.astype(np.int32))
for a, b, c in zip(a1, a2, ab):
    if a > 0:
        print(a,b,c)

3 0 3
2 1 3
3 0 3


In [ ]:
task_graph_end

In [18]:
len(cont['graph'])

102

In [20]:
cont['action'][0][100]

'[putback] <waterglass> (444) <kitchentable> (136)'

In [24]:
plate = [node['id'] for node in cont['graph'][0]['nodes'] if node['class_name'] == 'plate']

In [25]:
plate

[412, 413, 414, 415, 416]

In [27]:
[edge for edge in cont['graph'][-1]['edges'] if edge['from_id'] in plate and edge['to_id'] == 136]

[{'from_id': 414, 'to_id': 136, 'relation_type': 'ON'},
 {'from_id': 413, 'to_id': 136, 'relation_type': 'ON'},
 {'from_id': 416, 'to_id': 136, 'relation_type': 'ON'}]

In [57]:
[node for node in cont['graph'][0]['nodes'] if node['id'] == 132]

[{'id': 132,
  'category': 'Furniture',
  'class_name': 'kitchentable',
  'prefab_name': 'PRE_FUR_Kitchen_table_01_04',
  'obj_transform': {'position': [-3.498, 0.029, 1.632],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-3.498, 0.504, 1.632],
   'size': [1.092, 0.950000048, 2.0]},
  'properties': ['SURFACES', 'MOVABLE'],
  'states': []}]

In [53]:
[edge for edge in cont['graph'][0]['edges'] if edge['from_id'] in plate]

[{'from_id': 412, 'to_id': 11, 'relation_type': 'INSIDE'},
 {'from_id': 413, 'to_id': 11, 'relation_type': 'INSIDE'},
 {'from_id': 414, 'to_id': 11, 'relation_type': 'INSIDE'},
 {'from_id': 415, 'to_id': 11, 'relation_type': 'INSIDE'},
 {'from_id': 416, 'to_id': 11, 'relation_type': 'INSIDE'},
 {'from_id': 414, 'to_id': 132, 'relation_type': 'ON'},
 {'from_id': 413, 'to_id': 146, 'relation_type': 'INSIDE'},
 {'from_id': 412, 'to_id': 163, 'relation_type': 'INSIDE'},
 {'from_id': 415, 'to_id': 166, 'relation_type': 'INSIDE'},
 {'from_id': 416, 'to_id': 166, 'relation_type': 'INSIDE'}]

In [32]:
graph_info.keys()

dict_keys(['class_objects', 'edge_tuples', 'mask_edge', 'node_ids'])

In [36]:
import numpy as np
print(np.where(graph_info['node_ids'] == 414))
print(np.where(graph_info['node_ids'] == 413))
print(np.where(graph_info['node_ids'] == 416))
print(np.where(graph_info['node_ids'] == 136))

(array([17]),)
(array([16]),)
(array([19]),)
(array([2]),)


In [40]:
ind = (graph_info['edge_tuples'][:,1] == 2)
graph_info['edge_tuples'][ind]

array([[17,  2],
       [16,  2],
       [13,  2],
       [25,  2],
       [20,  2],
       [19,  2],
       [45,  2],
       [26,  2],
       [52,  2]], dtype=int32)